In [353]:
%pip install agentpy owlready2



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [354]:
import agentpy as ap
from matplotlib import pyplot as plt
import IPython
import numpy as np
from owlready2 import *
import random

In [355]:
onto = get_ontology("file://onto.owl")


In [356]:
#ONTOLOGIA
#onto.delete()
with onto:

    class Entity(Thing):
        pass

    class Vaccum(Entity):
        pass

    class Dirt(Entity):
        pass

    class Box(Entity):
        boxStack=1
        pass

    class Place(Thing):
        pass

    class Position(Thing):
        pass

    class has_place(ObjectProperty, FunctionalProperty):
        domain = [Entity]
        range = [Place]

    class has_position(DataProperty, FunctionalProperty):
        domain = [Place]
        range = [str]

        pass



In [357]:
class BoxPile(ap.Agent):

    def setup(self):
        self.boxStack = 1
        self.agentType = 4
        self.first_step = True
        self.pos = None
        self.stacked = False

    def add_box(self):
        if self.boxStack < 3:
            self.boxStack += 1
            self.stacked = True
        else:
            self.stacked = False
            print("Stack is full")

    def step(self):
        if self.first_step:
            self.pos = self.model.grid.positions[self]
            self.first_step = False
    def update(self):
        pass

    def end(self):
        pass

In [358]:
class Stack(ap.Agent):
    def setup(self):
        self.boxStack = 2
        self.agentType = 2  # New agent type for Stack

    def add_box(self):
        if self.boxStack < 5:
            self.boxStack += 1
            return True
        return False

    def remove_box(self):
        if self.boxStack > 2:
            self.boxStack -= 1
            return True
        return False
    def step(self):
        pass

    def update(self):
        pass
    
    def end(self):
        pass

In [359]:
import random

class Robot(ap.Agent):
    def setup(self):
        self.agentType = 0
        self.carryingBox = False
        self.direction = (0, -1)  # Initial direction (facing North)
        self.previous_position = None
        self.frustration = 0
        self.frustration_threshold = 5
        self.steps_since_last_turn = 0
        self.turn_interval = random.randint(5, 15)  # Randomize turn interval
        self.boxes_grabbed = 0
        self.boxes_stacked = 0
        self.actions = (
            self.move_and_grab,
            self.turn_and_stack,
            self.turn,
            self.random_turn,
            self.move_n,
            self.move_e,
            self.move_s,
            self.move_w,
            self.stack_box,  # New action
        )
        self.rules = (
            self.rule_move_and_grab,
            self.rule_turn_and_stack,
            self.rule_turn,
            self.rule_random_turn,
            self.rule_move_n,
            self.rule_move_e,
            self.rule_move_s,
            self.rule_move_w,
            self.rule_stack_box,  # New rule
        )
    def rule_stack_box(self, act):
        return act == self.stack_box and self.carryingBox and any(p[0] == "BoxPile" for p in self.per)


    def step(self):
        self.see(self.model.grid)
        self.next()

    def next(self):
        for act in self.actions:
            for rule in self.rules:
                if rule(act):
                    act()
                    return  # Only perform one action per step
        self.random_turn()  # If no action is taken, random turn

    def rule_move_and_grab(self, act):
        return act == self.move_and_grab

    def rule_turn_and_stack(self, act):
        return act == self.turn_and_stack and self.carryingBox and any(p[0] == "BoxPile" for p in self.per)

    def rule_turn(self, act):
        return act == self.turn

    def rule_random_turn(self, act):
        return act == self.random_turn and self.steps_since_last_turn >= self.turn_interval

    def rule_move_n(self, act):
        return act == self.move_n and not self.carryingBox

    def rule_move_e(self, act):
        return act == self.move_e and not self.carryingBox

    def rule_move_s(self, act):
        return act == self.move_s and not self.carryingBox

    def rule_move_w(self, act):
        return act == self.move_w and not self.carryingBox
    
    def stack_box(self):
        for perception in self.per:
            if perception[0] == "BoxPile":
                box_pos = perception[1]
                box_agents = self.model.grid.agents[box_pos]
                box_pile = next((agent for agent in box_agents if isinstance(agent, BoxPile)), None)
                if box_pile:
                    box_pile.add_box()
                    self.carryingBox = False
                    self.boxes_stacked += 1
                    print(f"Robot at {self.model.grid.positions[self]} stacked a box at {box_pos}. Stack height: {box_pile.boxStack}. Total stacked: {self.boxes_stacked}")
                    return
        print(f"Robot at {self.model.grid.positions[self]} couldn't find a box to stack on.")

    def move_and_grab(self):
        front_pos = self.get_front_position()
        if self.is_valid_position(front_pos):
            front_agents = self.model.grid.agents[front_pos]
            if not front_agents:
                # Move forward if the space is empty
                self.model.grid.move_by(self, self.direction)
                self.steps_since_last_turn += 1
                print(f"Robot at {self.model.grid.positions[self]} moved forward to {front_pos}")
            elif self.carryingBox and any(isinstance(agent, BoxPile) for agent in front_agents):
                # Stack the box if carrying one and colliding with another box
                self.stack_box()
            else:
                # Turn if there's an obstacle that's not a box
                self.random_turn()
                return
        else:
            # Turn if the position is invalid (border)
            self.random_turn()
            return

        # Check for adjacent boxes after moving
        if not self.carryingBox:
            for perception in self.per:
                if perception[0] == "BoxPile":
                    box_pos = perception[1]
                    box_agents = self.model.grid.agents[box_pos]
                    box = next((agent for agent in box_agents if isinstance(agent, BoxPile)), None)
                    if box and box.boxStack == 1:
                        self.model.grid.remove_agents(box)
                        self.model.boxes.remove(box)
                        self.carryingBox = True
                        self.boxes_grabbed += 1
                        print(f"Robot at {self.model.grid.positions[self]} grabbed an adjacent box from {box_pos}. Total grabbed: {self.boxes_grabbed}")
                        break


    def turn_and_stack(self):
        for perception in self.per:
            if perception[0] == "BoxPile":
                box_pos = perception[1]
                current_pos = self.model.grid.positions[self]
                direction = (box_pos[0] - current_pos[0], box_pos[1] - current_pos[1])
                
                # Turn until facing the box
                while self.direction != direction:
                    self.turn()
                
                # Now that we're facing the box, stack it
                box_agents = self.model.grid.agents[box_pos]
                box = next((agent for agent in box_agents if isinstance(agent, BoxPile)), None)
                if box:
                    # Instead of using a Stack agent, we can just mark the position as occupied
                    self.model.grid.remove_agents(box)
                    self.model.boxes.remove(box)
                    # Mark the position as occupied by a stack
                    self.model.grid.add_agents([], positions=[box_pos])  # No agent, just marking the position
                    self.carryingBox = False
                    print(f"Robot at {current_pos} created a stack at {box_pos}")
                return

    def move_n(self):
        self.direction = (-1, 0)  # North
        self.move()

    def move_e(self):
        self.direction = (0, 1)  # East
        self.move()

    def move_s(self):
        self.direction = (1, 0)  # South
        self.move()

    def move_w(self):
        self.direction = (0, -1)  # West
        self.move()

    def move(self):
        front_pos = self.get_front_position()
        if self.is_valid_position(front_pos):
            front_agents = self.model.grid.agents[front_pos]
            if not front_agents:
                # Move forward if the space is empty
                self.model.grid.move_by(self, self.direction)
                print(f"Robot at {self.model.grid.positions[self]} moved to {front_pos}")
            elif self.carryingBox and any(isinstance(agent, BoxPile) for agent in front_agents):
                # Stack the box if carrying one and colliding with another box
                self.stack_box()
            else:
                # Turn if there's an obstacle that's not a box
                self.random_turn()
        else:
            # Turn if the position is invalid (border)
            self.random_turn()


    def turn(self):
        if self.direction == (-1, 0):  # North
            self.direction = (0, 1)  # East
        elif self.direction == (0, 1):  # East
            self.direction = (1, 0)  # South
        elif self.direction == (1, 0):  # South
            self.direction = (0, -1)  # West
        elif self.direction == (0, -1):  # West
            self.direction = (-1, 0)  # North
        print(f"Robot at {self.model.grid.positions[self]} turned to face {self.direction}")

    def random_turn(self):
        directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]  # N, E, S, W
        self.direction = random.choice(directions)
        self.steps_since_last_turn = 0
        self.turn_interval = random.randint(5, 15)  # Reset turn interval
        print(f"Robot at {self.model.grid.positions[self]} randomly turned to face {self.direction}")

    def get_front_position(self):
        current_pos = self.model.grid.positions[self]
        return (current_pos[0] + self.direction[0], current_pos[1] + self.direction[1])

    def is_valid_position(self, pos):
        return 0 <= pos[0] < self.model.p.M and 0 <= pos[1] < self.model.p.N

    def see(self, grid):
        self.per = []
        current_pos = grid.positions[self]
        directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]  # N, E, S, W

        for direction in directions:
            neighbor_pos = (
                current_pos[0] + direction[0],
                current_pos[1] + direction[1],
            )

            if self.is_valid_position(neighbor_pos):
                agents_at_pos = grid.agents[neighbor_pos]
                if agents_at_pos:
                    agent = next(iter(agents_at_pos))
                    if isinstance(agent, BoxPile):
                        self.per.append(("BoxPile", neighbor_pos))
                    elif isinstance(agent, Robot):
                        self.per.append(("Robot", neighbor_pos))
                else:
                    self.per.append(("Empty", neighbor_pos))
            else:
                self.per.append(("Border", neighbor_pos))
    def end(self):
        print(f"Robot at {self.model.grid.positions[self]} final stats: Grabbed: {self.boxes_grabbed}, Stacked: {self.boxes_stacked}")

In [360]:
class RobotModel(ap.Model):
    def setup(self):

        self.robots = ap.AgentList(self, self.p.robots, Robot)
        self.boxes = ap.AgentList(self, self.p.boxes, BoxPile)
        self.stacks = ap.AgentList(self, 0, Stack)  # Start with 0 stacks

        self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)

        self.grid.add_agents(self.robots, random=True, empty=True)
        self.grid.add_agents(self.boxes, random=True, empty=True)

    def step(self):
        self.robots.step()

    def update(self):
        pass

    def end(self):
        print("\nSimulation Summary:")
        for i, robot in enumerate(self.robots):
            print(f"Robot {i}: Grabbed {robot.boxes_grabbed} boxes, Stacked {robot.boxes_stacked} boxes")
        
        stack_sizes = [box.boxStack for box in self.boxes if box.boxStack > 1]
        print(f"\nFinal stack count: {len(stack_sizes)}")
        print(f"Stack sizes: {stack_sizes}")
        print(f"Total boxes in stacks: {sum(stack_sizes)}")
        print(f"Single boxes remaining: {len([box for box in self.boxes if box.boxStack == 1])}")

In [361]:
parameters = {
    'M': 15,
    'N': 15,
    "steps": 300,
    "robots": 5,
    'boxes': 15
    
}

In [362]:
def animation_plot(model, ax):
    agent_type_grid = model.grid.attr_grid('agentType')
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    
    # Count the number of stacks (BoxPiles with more than one box)
    num_stacks = sum(1 for box in model.boxes if box.boxStack > 1)
    
    ax.set_title(f"Robot Model \n Time-step: {model.t}, "
                 f"Boxes: {len(model.boxes)}, Robots: {len(model.robots)}, "
                 f"Stacks: {num_stacks}", color='red')

In [363]:
#SIMULATION:

#Create figure (from matplotlib)
fig, ax = plt.subplots()

#Create model
model = RobotModel(parameters)

#Run with animation
#If you want to run it without animation then use instead:
#model.run()
animation = ap.animate(model, fig, ax, animation_plot)
#This step may take a while before you can see anything

#Print the final animation
IPython.display.HTML(animation.to_jshtml())



Robot at (6, 3) moved forward to (6, 3)
Robot at (5, 12) moved forward to (5, 12)
Robot at (8, 7) moved forward to (8, 7)
Robot at (14, 13) moved forward to (14, 13)
Robot at (12, 0) randomly turned to face (-1, 0)
Robot at (6, 2) moved forward to (6, 2)
Robot at (5, 11) moved forward to (5, 11)
Robot at (8, 6) moved forward to (8, 6)
Robot at (8, 6) grabbed an adjacent box from (9, 7). Total grabbed: 1
Robot at (14, 12) moved forward to (14, 12)
Robot at (11, 0) moved forward to (11, 0)
Robot at (6, 1) moved forward to (6, 1)
Robot at (5, 10) moved forward to (5, 10)
Robot at (8, 5) moved forward to (8, 5)
Robot at (14, 11) moved forward to (14, 11)
Robot at (10, 0) moved forward to (10, 0)
Robot at (6, 0) moved forward to (6, 0)
Robot at (6, 0) grabbed an adjacent box from (5, 1). Total grabbed: 1
Robot at (5, 9) moved forward to (5, 9)
Robot at (8, 4) moved forward to (8, 4)
Robot at (14, 10) moved forward to (14, 10)
Robot at (9, 0) moved forward to (9, 0)
Robot at (6, 0) randomly 